In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests

# Try to import Dash components, with fallback if not available
try:
    from dash import Dash, html, dcc, Input, Output
    import plotly.express as px
    DASH_AVAILABLE = True
except ImportError:
    print("Dash or Plotly not available. Running in fallback mode.")
    DASH_AVAILABLE = False

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"

def load_df():
    try:
        df = pd.read_csv(URL, parse_dates=['Date'])
        print("Loaded CSV")
    except:
        print("Falling back to synthetic dataset")
        years = np.arange(1978, 2025)
        months = np.arange(1, 13)
        vts = ["Supperminicar", "Smallfamiliycar", "Mediumfamilycar", "Executivecar", "Sports"]
        rec_windows = [((1980,1),(1980,12)), ((1981,1),(1982,12)), ((1991,1),(1991,12)),
                      ((2000,1),(2001,12)), ((2007,12),(2009,6)), ((2020,2),(2020,4))]
        rows = []
        
        for y in years:
            for m in months:
                for vt in vts:
                    in_rec = any((y > sy or (y == sy and m >= sm)) and (y < ey or (y == ey and m <= em)) 
                                for (sy, sm), (ey, em) in rec_windows)
                    sales = max(0, (800 + 80 * vts.index(vt)) * 
                              (1 + 0.12 * np.sin((m-1)/12 * 2 * np.pi)) *
                              (1 + 0.015 * (y-1978)) * 
                              (0.75 if in_rec else 1.03) * 
                              (1 + np.random.normal(0, 0.08)))
                    price = 20000 + 100 * (y-1978)
                    adv_multipliers = {
                        'Supperminicar': 0.6,
                        'Smallfamiliycar': 0.9,
                        'Mediumfamilycar': 1.0,
                        'Executivecar': 0.8,
                        'Sports': 0.5
                    }
                    adv = 150000 * adv_multipliers[vt] * (1.1 if in_rec else 1.0)
                    gdp = 10000 + 220 * (y-1978) - (800 if in_rec else 0) + 80 * np.random.randn()
                    unemp = max(0.5, 4 + (1.8 if in_rec else 0.3) + 0.6 * np.random.randn())
                    
                    rows.append({
                        'Date': pd.Timestamp(year=y, month=m, day=1),
                        'Recession': int(in_rec),
                        'Automobile_Sales': sales,
                        'GDP': gdp,
                        'Unemployment_Rate': unemp,
                        'Price': price,
                        'Advertising_Expenditure': adv,
                        'Vehicle_Type': vt,
                        'Month': m,
                        'Year': y
                    })
        df = pd.DataFrame(rows)
    
    if 'Month' not in df.columns: 
        df['Month'] = df['Date'].dt.month
    if 'Year' not in df.columns: 
        df['Year'] = df['Date'].dt.year
        
    return df

df = load_df()

# Only create Dash app if dependencies are available
if DASH_AVAILABLE:
    app = Dash(__name__)
    app.title = "XYZAutomotives — Recession & Yearly Dashboard"

    vehicle_options = [{'label': v, 'value': v} for v in sorted(df['Vehicle_Type'].unique())]
    report_options = [
        {'label': 'Recession Report', 'value': 'recession'},
        {'label': 'Yearly Report', 'value': 'yearly'}
    ]

    app.layout = html.Div([
        html.H1(app.title),
        html.Div([
            dcc.Dropdown(
                id='vehicle-dropdown', 
                options=vehicle_options, 
                value=vehicle_options[0]['value'], 
                clearable=False
            ),
            dcc.Dropdown(
                id='report-dropdown', 
                options=report_options, 
                value='recession', 
                clearable=False
            )
        ], style={'display': 'flex', 'gap': '12px', 'width': '80%'}),
        html.Div(id='report-output', className='report-box'),
        dcc.Graph(id='main-graph')
    ], style={'padding': 20})

    @app.callback(
        [Output('report-output', 'children'), Output('main-graph', 'figure')],
        [Input('vehicle-dropdown', 'value'), Input('report-dropdown', 'value')]
    )
    def update_report(vehicle, report_type):
        if report_type == 'recession':
            d = df[df['Recession'] == 1]
            avg_monthly = d[d['Vehicle_Type'] == vehicle]['Automobile_Sales'].mean()
            summary = html.Div([html.P(f"Recession report for {vehicle}: avg monthly sales = {avg_monthly:.0f}")])
            agg = d.groupby('Year', as_index=False)['Automobile_Sales'].mean()
            fig = px.line(agg, x='Year', y='Automobile_Sales', title=f"{vehicle} Avg Sales across Recession Years")
        else:
            d = df
            yearly = d[d['Vehicle_Type'] == vehicle].groupby('Year', as_index=False)['Automobile_Sales'].sum()
            avg_yearly = yearly['Automobile_Sales'].mean()
            summary = html.Div([html.P(f"Yearly report for {vehicle}: avg yearly sales = {avg_yearly:.0f}")])
            fig = px.bar(yearly, x='Year', y='Automobile_Sales', title=f"{vehicle} Yearly Sales")
        return summary, fig

# Save static helper images
fig = plt.figure(figsize=(8, 2))
plt.text(0.5, 0.5, "XYZAutomotives — Recession & Yearly Dashboard", 
         horizontalalignment='center', verticalalignment='center', fontsize=16)
plt.axis('off')
fig.savefig('Title.png', dpi=150)
plt.close()

fig = plt.figure(figsize=(8, 2))
plt.text(0.02, 0.7, "Dropdowns: vehicle, report", fontsize=10)
plt.axis('off')
fig.savefig('Dropdown.png', dpi=150)
plt.close()

fig = plt.figure(figsize=(8, 2))
plt.text(0.02, 0.5, 'Output DIV (id="report-output", className="report-box")', fontsize=10)
plt.axis('off')
fig.savefig('outputdiv.png', dpi=150)
plt.close()

fig = plt.figure(figsize=(8, 3))
plt.text(0.01, 0.8, 'Callbacks: vehicle-dropdown + report-dropdown -> report-output + main-graph', fontsize=10)
plt.axis('off')
fig.savefig('Callbacks.png', dpi=150)
plt.close()

# Create sample plots using matplotlib instead of plotly
rec_summary = df[df['Recession'] == 1].groupby('Vehicle_Type', as_index=False)['Automobile_Sales'].sum()
plt.figure(figsize=(10, 6))
plt.bar(rec_summary['Vehicle_Type'], rec_summary['Automobile_Sales'])
plt.title('Recession Report: Sales by Vehicle Type')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('RecessionReportgraphs.png', dpi=150)
plt.close()

yearly_summary = df.groupby('Year', as_index=False)['Automobile_Sales'].sum()
plt.figure(figsize=(10, 6))
plt.plot(yearly_summary['Year'], yearly_summary['Automobile_Sales'], marker='o')
plt.title('Yearly Report: Total Sales')
plt.tight_layout()
plt.savefig('YearlyReportgraphs.png', dpi=150)
plt.close()

print("Saved Title.png, Dropdown.png, outputdiv.png, Callbacks.png")
print("Saved RecessionReportgraphs.png and YearlyReportgraphs.png")

# Only run the Dash server if dependencies are available
if DASH_AVAILABLE:
    if __name__ == '__main__':
        app.run_server(debug=False)
else:
    print("Dash is not available. Install it with: pip install dash plotly")

Dash or Plotly not available. Running in fallback mode.
Loaded CSV
Saved Title.png, Dropdown.png, outputdiv.png, Callbacks.png
Saved RecessionReportgraphs.png and YearlyReportgraphs.png
Dash is not available. Install it with: pip install dash plotly
